### Split-operator 2D GPE(Gross-Pitaevskii Equation) solver: 
Application of FFT

Inspired from below article.

https://www.algorithm-archive.org/contents/split-operator_method/split-operator_method.html

Gross-Pitaevskii Equation describe bosons systems wavefunction.

Bosons allow for multiple particles to occupy the same state simultaneously due to the absence of the exclusion principle. Therefore, the GPE models the distribution of such a system as a **single wave function**.

$$i\hbar \frac{\partial \Psi(\mathbf{r}, t)}{\partial t} = \left[ -\frac{\hbar^2}{2m}\nabla^2 + V(\mathbf{r}) + g|\Psi(\mathbf{r}, t)|^2 \right] \Psi(\mathbf{r}, t)$$

This is a kind of NLSE. Factor g describe 'interaction of bosons'.
$$g = \frac{4\pi\hbar^2 a_s}{m}$$

Instead of this 'energy density' coeffcient,
we actually use 'dimless' coefficient when we implement computer code.

$$\tilde{g} \approx \frac{4\pi N a_s}{a_{ho}}$$

We can split Hamlitonian as momentum part and radial part.

$$i\hbar \frac{\partial \Psi}{\partial t} = \hat{H} \Psi = (\hat{H}_k + \hat{H}_r) \Psi$$

where

$$\hat{H}_k = -\frac{\hbar^2}{2m}\nabla^2, \hat{H}_r = V(\mathbf{r}) + g|\Psi(\mathbf{r}, t)|^2$$

Short time evolution of the wave function can be described as exponential form

$$\Psi(t+\Delta t) = e^{-\frac{i}{\hbar}\hat{H}\Delta t} \Psi(t) = e^{-\frac{i}{\hbar}(\hat{H}_k + \hat{H}_r)\Delta t} \Psi(t)$$

Let's set
$\hat{A} = -i\hat{H}_r dt$, $\hat{B} = -i\hat{H}_k dt$

We can simplify time evolution formula using BCH(Baker-Campbell-Hausdorff Formula)

$$e^{\hat{A}} e^{\hat{B}} = \exp\left( \hat{A} + \hat{B} + \frac{1}{2}[\hat{A}, \hat{B}] \right)$$

This formula has error $O(\Delta t^2)$.
To minimize the error, we can use Strang splitting


$0 to (\Delta t)/2$ error is $\frac{1}{8}[\hat{A}, \hat{B}]$.

$(\Delta t)/2 to (\Delta t)$ error is $\frac{1}{8}[\hat{B}, \hat{A}]$.

Error $O(\Delta t^2)$ is cancelld!

So, we can describe time evolution

$$e^{-i(\hat{H}_k + \hat{H}_r)\Delta t} \approx e^{-i\hat{H}_r \frac{\Delta t}{2}} e^{-i\hat{H}_k \Delta t} e^{-i\hat{H}_r \frac{\Delta t}{2}} + O(\Delta t^3)$$

We can proceed with this calculation, but there is a specific complication. While the calculation for the radial part (potential term) of the Hamiltonian is straightforward and can be performed simply as a multiplication of functions, the momentum part requires evaluating the exponential of a differential operator.

This difficulty can be overcome using the Fast Fourier Transform (FFT). By computing the radial part in real space and the momentum part in momentum space, the process becomes much simpler. This approach is known as the split-operator method.

$$\Psi(t+\Delta t) \approx \underbrace{\hat{U}_r\left(\frac{\Delta t}{2}\right)}_{\text{Half-kick}} \cdot \underbrace{\mathcal{F}^{-1} \left[ \hat{U}_k(\Delta t) \cdot \mathcal{F} \left[ \hat{U}_r\left(\frac{\Delta t}{2}\right) \Psi(t) \right] \right]}_{\text{Drift in k-space}}$$

where
$$\hat{U}_k(\Delta t) = e^{-i \hat{H}_k \Delta t} = e^{-i \left(-\frac{\nabla^2}{2m}\right) \Delta t}, \hat{U}_r(\Delta t) = e^{-i \hat{H}_r \Delta t} = e^{-i \left(V + g|\Psi|^2\right) \Delta t}$$

By iterating this calculation, we can compute the time evolution over long periods. This allows us to simulate the dynamics of the wave function. Let us examine the motion of the wave function in two specific potentials: the Simple Harmonic Oscillator (SHO) and the Double-Well potential.

Next, let us substitute the time parameter $t$ with the imaginary time parameter $\tau = it$. Consequently, the imaginary unit $i$ in the exponents of the operators disappears, transforming them into real exponential functions. As $\tau$ accumulates, higher energy terms decay exponentially, leaving only the term corresponding to the lowest energy eigenvalue (the ground state). In other words, the result of this propagation converges to the ground state, regardless of the initial state.Based on this imaginary time evolution, we can obtain the density distribution of the ground state. This corresponds to the approximate form of a Bose-Einstein Condensate (BEC).

We will verify how the ground state distribution changes with respect to the magnitude of the repulsive interaction strength $g$ and the initial wave function, and compare these numerical results with the analytical Thomas-Fermi approximation.

### Thomas-Fermi Approximation

The Thomas-Fermi (TF) approximation provides an analytical solution for the ground state of a Bose-Einstein Condensate (BEC) in the limit of strong interactions (large $g$).

#### 1. Time-Independent GPE
We start with the time-independent Gross-Pitaevskii Equation (GPE):
$$
\mu \psi(\mathbf{r}) = \left[ -\frac{\hbar^2}{2m}\nabla^2 + V(\mathbf{r}) + g|\psi(\mathbf{r})|^2 \right] \psi(\mathbf{r})
$$
Where $\mu$ is the chemical potential.

#### 2. The Thomas-Fermi Approximation
In the regime where the interaction energy ($g|\psi|^2$) is much larger than the kinetic energy, we can neglect the kinetic energy term ($-\frac{\hbar^2}{2m}\nabla^2$). This is known as the Thomas-Fermi limit.

Ignoring the kinetic term, the equation simplifies to an algebraic equation:
$$
\mu \psi(\mathbf{r}) \approx \left[ V(\mathbf{r}) + g|\psi(\mathbf{r})|^2 \right] \psi(\mathbf{r})
$$

#### 3. Density Profile
Solving for the density $n(\mathbf{r}) = |\psi(\mathbf{r})|^2$:
$$
\mu = V(\mathbf{r}) + g n(\mathbf{r})
$$
$$
n_{TF}(\mathbf{r}) = \frac{\mu - V(\mathbf{r})}{g}
$$
Since the density must be non-negative, the Thomas-Fermi density profile is given by:
$$
n_{TF}(\mathbf{r}) = \max\left( 0, \frac{\mu - V(\mathbf{r})}{g} \right)
$$
This describes an "inverted parabola" shape in a harmonic trap.

#### 4. Chemical Potential ($\mu$) for 2D SHO
For a 2D isotropic harmonic oscillator, $V(r) = \frac{1}{2}m\omega^2 r^2$. The chemical potential $\mu$ is determined by the normalization condition $\int n(\mathbf{r}) d^2\mathbf{r} = N$. Assuming $N=1$:

$$
\mu_{2D} = \sqrt{\frac{g m \omega^2}{\pi}}
$$

This analytical result serves as a benchmark for validating the accuracy of the numerical GPE solver.

For intuitive visuallization, my implementation was done on 2D.

Import modules

In [1]:
# plt를 임포트하기 전에 백엔드를 TkAgg로 강제. 시각화 윈도우.
import matplotlib
matplotlib.use('TkAgg')

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fftn, ifftn, fftfreq
from matplotlib.animation import FuncAnimation

# 자연 단위계 (Natural Units) 설정
# hbar = 1
# m = 1 (필요시 클래스 매개변수로 조절 가능)

Real space, Momentum space generator

In [2]:
# --- (1) Param2D 클래스 (k-그리드 fftfreq 방식으로 수정) ---
class Param2D:
    """2D 시뮬레이션 매개변수 (k-그리드 수정됨)"""
    
    def __init__(self, xmax=10.0, res=128, dt=0.01, timesteps=1000, m=1.0, 
                 im_time=False):
        
        self.xmax = xmax
        self.res = res
        self.dt = dt
        self.timesteps = timesteps
        self.m = m
        self.im_time = im_time
        
        # 1D 실 공간 그리드 (linspace 사용이 더 안정적)
        self.dx = 2 * self.xmax / self.res
        self.x = np.linspace(-self.xmax, self.xmax - self.dx, self.res)
        self.y = self.x

        # 2D 실 공간 그리드 (Meshgrid)
        self.X, self.Y = np.meshgrid(self.x, self.y, indexing='ij')

        # 1D & 2D 운동량 그리드 (fftfreq 사용)
        k_freq_1d = fftfreq(self.res, self.dx)
        self.kx_1d = 2 * np.pi * k_freq_1d
        self.ky_1d = 2 * np.pi * k_freq_1d
        
        self.Kx, self.Ky = np.meshgrid(self.kx_1d, self.ky_1d, indexing='ij')
        self.K_sq = self.Kx**2 + self.Ky**2

Define potentials

Simple Harmonic Oscillator potential

$$V_{SHO}(x, y) = \frac{1}{2}m\omega^2 (x^2 + y^2)$$

Double-well potential

$$V_{DW}(x, y) = \underbrace{\frac{1}{2}m\omega^2 (x^2 + y^2)}_{\text{Harmonic Trap}} + \underbrace{V_0 \exp\left(-\frac{x^2}{2\sigma^2}\right)}_{\text{Gaussian Barrier}}$$

In [3]:
# --- (2) 퍼텐셜 함수 ---
def potential_sho(par: Param2D, omega=1.0):
    """2D 조화 진동자(SHO) 퍼텐셜"""
    return 0.5 * par.m * (omega**2) * (par.X**2 + par.Y**2)

def potential_double_well(par: Param2D, omega=1.0, barrier_height=40.0, barrier_width=0.5):
    """
    2D 이중 우물 퍼텐셜 (Double-Well Potential)
    : 전체적으로 조화 진동자(SHO)로 가두고, 중앙(x=0)에 가우시안 장벽을 세웁니다.
    """
    # 1. 원자 트랩 (SHO)
    V_trap = 0.5 * par.m * (omega**2) * (par.X**2 + par.Y**2)
    
    # 2. 중앙을 가로막는 장벽 (Barrier)
    # x=0 위치에 솟아오른 가우시안 언덕 생성.
    V_barrier = barrier_height * np.exp(-par.X**2 / (2 * barrier_width**2))
    
    # 혼합 장벽이 이중 우물
    return V_trap + V_barrier

Initial wave functions

Gaussian
$$\Psi_{\text{Gaussian}}(x, y) = A \exp\left(-\frac{(x-x_0)^2 + (y-y_0)^2}{2}\right)$$

Noise
$$\Psi_{\text{Random}}(x, y) = A \left[ \mathcal{R}(x,y) + i \mathcal{I}(x,y) \right] \times \exp\left(-\frac{x^2 + y^2}{x_{\max}^2}\right)$$

Square
$$\Psi_{\text{Square}}(x, y) = \begin{cases} A & \text{if } |x| < w \text{ and } |y| < w \\ 0 & \text{otherwise} \end{cases}$$

Normalization condition
$$\int_{-\infty}^{\infty} \int_{-\infty}^{\infty} |\Psi(x, y)|^2 \, dx \, dy = 1$$

Momentum part of Hamiltonian(in momentum space)
$$\hat{H}_k = \frac{\hbar^2 k^2}{2m} \xrightarrow{\hbar=1} \frac{k_x^2 + k_y^2}{2m}$$

Real time evolution(im_time = False)
$$\hat{U}_k(\Delta t) = \exp\left(-i \hat{H}_k \Delta t\right) = \exp\left(-i \frac{k^2}{2m} \Delta t\right)$$

Imaginary time evolution(im_time = True, $\tau = it$)
$$\hat{U}_k(\Delta t) = \exp\left(-\hat{H}_k \Delta t\right) = \exp\left(-\frac{k^2}{2m} \Delta t\right)$$

In [4]:
# --- (3) 초기화 함수 ---
def init_state_and_ops(par: Param2D, wfc_offset=(0.0, 0.0), g=0.0, initial_type='gaussian'):
    """
    초기 파동함수(wfc)를 다양한 형태로 초기화합니다.
    - initial_type='gaussian': 기존 가우시안
    - initial_type='random': 완전 무작위 노이즈 (추천!)
    - initial_type='square': 네모난 상자 모양
    """
    
    # 1. 파동함수 모양 결정
    if initial_type == 'gaussian':
        x0, y0 = wfc_offset
        wfc = np.exp(-((par.X - x0)**2 + (par.Y - y0)**2) / 2.0)
        wfc = wfc.astype(complex)
        
    elif initial_type == 'random':
        # 0~1 사이의 난수로 채웁니다. (실수부 + 허수부)
        # 시드(seed)를 고정하면 매번 같은 노이즈가 나옵니다.
        np.random.seed(42) 
        real_part = np.random.rand(par.res, par.res)
        imag_part = np.random.rand(par.res, par.res)
        wfc = real_part + 1j * imag_part
        
        # 경계 조건 문제 방지를 위해 가장자리는 0으로 깎아줍니다 (윈도우 함수 적용)
        # (선택 사항이지만 안정성을 위해 추천)
        window = np.exp(-(par.X**2 + par.Y**2) / (par.xmax**2))
        wfc *= window

    elif initial_type == 'square':
        # 중심에 있는 네모난 상자 모양 (계단 함수)
        width = 2.0
        mask = (np.abs(par.X) < width) & (np.abs(par.Y) < width)
        wfc = np.zeros_like(par.X, dtype=complex)
        wfc[mask] = 1.0

    else:
        raise ValueError("Unknown initial_type")

    # 2. 규격화 (필수!)
    # 노이즈나 네모 모양은 규격화가 안 되어 있으므로 반드시 해야 합니다.
    wfc_norm = np.sqrt(np.sum(np.abs(wfc)**2) * par.dx * par.dx)
    wfc /= wfc_norm

    # 3. 운동량 연산자 및 g 설정 (기존과 동일)
    Hk_kspace = (par.K_sq) / (2 * par.m)
    
    if par.im_time:
        Uk_full = np.exp(-Hk_kspace * par.dt)
    else:
        Uk_full = np.exp(-1j * Hk_kspace * par.dt)

    return wfc, Uk_full, g

$$\Psi(t+\Delta t) = e^{-\frac{i}{\hbar}\hat{H}\Delta t} \Psi(t) = e^{-\frac{i}{\hbar}(\hat{H}_k + \hat{H}_r)\Delta t} \Psi(t)$$

In [5]:
# --- (4) GPE 스텝 함수 ---
def split_op_2d_step(par: Param2D, wfc: np.ndarray, Uk_full: np.ndarray, V: np.ndarray, g: float):
    """2D GPE 스트랭 분할의 *단 한 스텝*을 수행합니다."""
    # (실수 시간 GPE를 가정하고 작성)
    Hr = V + g * np.abs(wfc)**2
    Ur_half = np.exp(-1j * Hr * par.dt / 2.0)
    wfc = wfc * Ur_half
    
    wfc = fftn(wfc)
    wfc = wfc * Uk_full
    wfc = ifftn(wfc)
    
    Hr = V + g * np.abs(wfc)**2
    Ur_half = np.exp(-1j * Hr * par.dt / 2.0)
    wfc = wfc * Ur_half
    
    # (허수 시간 로직 - 필요시 주석 해제 및 수정)
    if par.im_time:
       Hr = V + g * np.abs(wfc)**2
       Ur_half = np.exp(-Hr * par.dt / 2.0)
       wfc = wfc * Ur_half
       wfc = fftn(wfc)
       wfc = wfc * Uk_full # Uk_full은 init에서 im_time=True로 생성되어야 함
       wfc = ifftn(wfc)
       Hr = V + g * np.abs(wfc)**2
       Ur_half = np.exp(-Hr * par.dt / 2.0)
       wfc = wfc * Ur_half
       # 재규격화
       wfc_norm = np.sqrt(np.sum(np.abs(wfc)**2) * par.dx * par.dx)
       if wfc_norm > 1e-9: # 0으로 나누기 방지
           wfc /= wfc_norm
        
    return wfc

2D visualization

In [6]:
# --- (5) (수정) 재규격화가 추가된 FuncAnimation 실행 ---
def run_simulation(par: Param2D, wfc_initial: np.ndarray, Uk_full: np.ndarray, V: np.ndarray, g: float):
    """
    FuncAnimation을 사용하여 전체 시뮬레이 V (허수 시간 재규격화 추가)
    """
    
    fig, ax = plt.subplots()
    density = np.abs(wfc_initial)**2
    
    density_plot = ax.imshow(density.T,
                             extent=[par.x[0], par.x[-1], par.y[0], par.y[-1]],
                             origin='lower', aspect='auto', interpolation='nearest',
                             vmin=0, vmax=np.max(density))
                             
    fig.colorbar(density_plot, ax=ax)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    
    wfc_container = [wfc_initial.copy()]
    
    steps_per_frame = 50 # 50배속 유지

    # --- 애니메이션의 각 프레임을 갱신하는 함수 ---
    def update(i):
        wfc = wfc_container[0]
        
        for _ in range(steps_per_frame):
            wfc = split_op_2d_step(par, wfc, Uk_full, V, g)
            
            # --- (중요) 허수 시간 재규격화 ---
            if par.im_time:
                wfc_norm = np.sqrt(np.sum(np.abs(wfc)**2) * par.dx * par.dx)
                if wfc_norm > 1e-9: # 0으로 나누기 방지
                    wfc /= wfc_norm
            # --------------------------------

            if np.isnan(wfc).any():
                print(f"!!! 시뮬레이션이 Time Step {i * steps_per_frame}에서 'NaN'으로 폭발했습니다. !!!")
                ani.event_source.stop()
                return

        wfc_container[0] = wfc
        
        density = np.abs(wfc)**2
        density_plot.set_data(density.T)
        density_plot.set_clim(vmin=0, vmax=np.max(density))
        ax.set_title(f"Time Step: {i * steps_per_frame}" f" g={g}") 

        if i % (100 // steps_per_frame) == 0: 
            print(f"Running frame {i} (Physics step {i * steps_per_frame})")

    total_frames = par.timesteps // steps_per_frame
    
    ani = FuncAnimation(fig,
                        update,
                        frames=total_frames,
                        interval=1,
                        blit=False)
                        
    plt.show() 
    
    return wfc_container[0]

SHO realtime simulation

In [7]:
# --- (6) (수정) 50배속 및 2D 오프셋을 적용한 main 함수 ---
def main():
    """메인 실행 함수"""
    
    print("2D GPE 시뮬레이션 (SHO Sloshing, 50x speed) 시작...")
    
    # --- 1. 파라미터 설정 ---
    # timesteps=5000 (50의 배수), dt=0.001 (안정)
    par = Param2D(xmax=5.0, res=128, dt=0.001, timesteps=5000, m=1.0, im_time=False)
    
    # --- 2. 퍼텐셜 V 정의 ---
    V = potential_sho(par, omega=1.0)
    
    # --- 3. 초기 상태 및 Uk 정의 ---
    g = 0.0 # g=0 (BEC 상호작용 없음)
    
    # (수정) 파동함수 중심을 (2.0, 2.0)으로 이동
    wfc_offset = (2.0, 2.0) 
    
    wfc_initial, Uk_full, g = init_state_and_ops(par, wfc_offset, g)
    
    # --- 4. 시뮬레이션 실행 ---
    wfc_final = run_simulation(par, wfc_initial, Uk_full, V, g)
    # --- 동영상 저장 ---
    

    print("시뮬레이션 완료.")

# 이 스크립트가 직접 실행될 때만 main() 함수를 호출합니다.
if __name__ == "__main__":
    main()

2D GPE 시뮬레이션 (SHO Sloshing, 50x speed) 시작...
Running frame 0 (Physics step 0)
Running frame 0 (Physics step 0)
Running frame 2 (Physics step 100)
Running frame 4 (Physics step 200)
Running frame 6 (Physics step 300)
Running frame 8 (Physics step 400)
Running frame 10 (Physics step 500)
Running frame 12 (Physics step 600)
Running frame 14 (Physics step 700)
Running frame 16 (Physics step 800)
Running frame 18 (Physics step 900)
Running frame 20 (Physics step 1000)
Running frame 22 (Physics step 1100)
Running frame 24 (Physics step 1200)
Running frame 26 (Physics step 1300)
Running frame 28 (Physics step 1400)
Running frame 30 (Physics step 1500)
Running frame 32 (Physics step 1600)
Running frame 34 (Physics step 1700)
Running frame 36 (Physics step 1800)
시뮬레이션 완료.


TF approximation comparison(when imaginary time evolutoin)

In [8]:
def calculate_and_plot_tf_comparison(par: Param2D, wfc_final: np.ndarray, V: np.ndarray, g: float):
    """
    최종 GPE 해(FFT)와 토마스-페르미(TF) 이론 해를 1D 그래프로 비교하고,
    정량적인 일치도(RMSE, 상대 오차)를 계산합니다.
    """
    print("\n--- 정량 분석 시작: 토마스-페르미(TF) 비교 ---")
    
    # 1. GPE(FFT) 해에서 1D 데이터 추출 (중앙 슬라이스 y=0)
    center_idx = par.res // 2
    density_fft = np.abs(wfc_final[center_idx, :])**2
    x_axis = par.x
    
    # 2. TF 이론 해 계산 및 정량 비교
    omega = 1.0 
    rmse = 0.0
    rel_error = 0.0
    
    if g > 0:
        # TF 근사 화학 퍼텐셜
        mu = np.sqrt(g * par.m * (omega**2) / np.pi)
        
        # 1D 퍼텐셜 슬라이스 (y=0)
        V_1d_slice = V[center_idx, :]
        
        # TF 밀도 공식
        density_tf = (mu - V_1d_slice) / g
        density_tf = np.maximum(0, density_tf) # 음수 값은 0으로 처리
        
        # --- [추가된 부분] 정량적 오차 계산 ---
        
        # (1) RMSE (평균 제곱근 오차): 절대적인 오차 크기
        # 두 곡선 사이의 평균적인 거리
        mse = np.mean((density_fft - density_tf)**2)
        rmse = np.sqrt(mse)
        
        # (2) Relative Error (상대 오차, L2 Norm): 전체 크기 대비 오차 비율
        # || n_sim - n_tf || / || n_tf ||
        diff_norm = np.linalg.norm(density_fft - density_tf)
        tf_norm = np.linalg.norm(density_tf)
        
        if tf_norm > 0:
            rel_error = diff_norm / tf_norm
        
        print(f"  >> 분석 결과 (g={g}):")
        print(f"  >> RMSE (절대 오차): {rmse:.6f}")
        print(f"  >> Relative Error (불일치도): {rel_error * 100:.4f} %")
        print(f"  >> Consistency (일치도): {(1 - rel_error) * 100:.4f} %")
        
    else:
        density_tf = np.zeros_like(x_axis)
        print("Warning: g=0 이므로 TF 근사를 적용할 수 없습니다.")

    # 3. 그래프 그리기
    plt.figure(figsize=(8, 6))
    plt.plot(x_axis, density_fft, 'b.', label='Simulation (FFT)', markersize=5, alpha=0.6)
    
    if g > 0:
        plt.plot(x_axis, density_tf, 'r-', label='Thomas-Fermi Theory', linewidth=2, alpha=0.8)
        
        # 그래프 제목에 정량 지표 추가
        title_str = (f"Quantitative Comparison (g={g})\n"
                     f"Rel. Error: {rel_error*100:.2f}% (Match: {(1-rel_error)*100:.1f}%)")
        plt.title(title_str)
        
        # 그래프 내부에 텍스트로도 표시 (선택 사항)
        stats_text = f"RMSE: {rmse:.4f}\nRel. Err: {rel_error*100:.2f}%"
        plt.text(0.05, 0.95, stats_text, transform=plt.gca().transAxes, 
                 fontsize=10, verticalalignment='top', 
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    else:
        plt.title("Simulation Result (g=0, No TF Approx)")

    plt.xlabel("Position x (at y=0)")
    plt.ylabel("Density |ψ(x, 0)|²")
    plt.legend(loc='upper right')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
    
    print("--- 정량 분석 완료 ---")

SHO imaginary time simulation

In [9]:
# --- (6) (수정) 허수 시간(바닥 상태)을 위한 main 함수 ---
def main2():
    """메인 실행 함수"""
    
    print("2D GPE 시뮬레이션 (Imaginary Time Ground State) 시작...")
    
    # --- 1. 파라미터 수정 ---
    # (중요) im_time=True로 변경
    par = Param2D(xmax=12.0, res=128, dt=0.001, timesteps=5000, m=1.0, im_time=True)
    
    # --- 2. 퍼텐셜 V 정의 ---
    V = potential_sho(par, omega=1.0)
    
    # --- 3. 초기 상태 및 Uk 정의 ---
    # (중요) g=500 으로 설정하여 BEC 상호작용(반발력)을 켭니다.
    # 0.001에서 잘 안맞음
    # 50000 커서 경향성깨짐
    g = 500
    
    # (2.0, 2.0)에서 시작해도 허수 시간 전개는 중심으로 수렴합니다.
    wfc_offset = (2.0, 2.0) 
    
    wfc_initial, Uk_full, g = init_state_and_ops(par, wfc_offset, g)
    
    # --- 4. 시뮬레이션 실행 ---
    wfc_final = run_simulation(par, wfc_initial, Uk_full, V, g)
    
    # 시뮬레이션이 끝난 후 정량 비교 실행
    if par.im_time: # 허수 시간일 때만 수행
        calculate_and_plot_tf_comparison(par, wfc_final, V, g)
    # ==============================================

    print("시뮬레이션 완료.")

# 이 스크립트가 직접 실행될 때만 main() 함수를 호출합니다.
if __name__ == "__main__":
    main2()

2D GPE 시뮬레이션 (Imaginary Time Ground State) 시작...
Running frame 0 (Physics step 0)
Running frame 0 (Physics step 0)
Running frame 2 (Physics step 100)
Running frame 4 (Physics step 200)
Running frame 6 (Physics step 300)
Running frame 8 (Physics step 400)
Running frame 10 (Physics step 500)
Running frame 12 (Physics step 600)
Running frame 14 (Physics step 700)

--- 정량 분석 시작: 토마스-페르미(TF) 비교 ---
  >> 분석 결과 (g=500):
  >> RMSE (절대 오차): 0.000802
  >> Relative Error (불일치도): 6.7287 %
  >> Consistency (일치도): 93.2713 %
--- 정량 분석 완료 ---
시뮬레이션 완료.


Noise imaginary time simulation

In [10]:
def main3():
    print("2D GPE 시뮬레이션 (Imaginary Time - Random Start) 시작...")
    
    # 1. 파라미터 (기존과 동일)
    par = Param2D(xmax=12.0, res=128, dt=0.001, timesteps=5000, m=1.0, im_time=True)
    V = potential_sho(par, omega=1.0)
    g = 500.0 
    
    # 2. 초기화
    wfc_offset = (0.0, 0.0) 
    
    # 'random' 모드로 초기화하여 "무질서에서 질서가 생기는 과정"을 봅니다.
    wfc_initial, Uk_full, g = init_state_and_ops(par, wfc_offset, g, initial_type='random')
    
    # 3. 시뮬레이션 실행
    wfc_final = run_simulation(par, wfc_initial, Uk_full, V, g)
    
    # 4. 정량 비교
    if par.im_time:
        calculate_and_plot_tf_comparison(par, wfc_final, V, g)
    
    print("시뮬레이션 완료.")

if __name__ == "__main__":
    main3()

2D GPE 시뮬레이션 (Imaginary Time - Random Start) 시작...
Running frame 0 (Physics step 0)
Running frame 0 (Physics step 0)
Running frame 2 (Physics step 100)
Running frame 4 (Physics step 200)
Running frame 6 (Physics step 300)
Running frame 8 (Physics step 400)
Running frame 10 (Physics step 500)
Running frame 12 (Physics step 600)
Running frame 14 (Physics step 700)
Running frame 16 (Physics step 800)
Running frame 18 (Physics step 900)

--- 정량 분석 시작: 토마스-페르미(TF) 비교 ---
  >> 분석 결과 (g=500.0):
  >> RMSE (절대 오차): 0.000267
  >> Relative Error (불일치도): 2.2410 %
  >> Consistency (일치도): 97.7590 %
--- 정량 분석 완료 ---
시뮬레이션 완료.


Square imaginary simulation

In [11]:
def main4():
    print("2D GPE 시뮬레이션 (Imaginary Time - Random Start) 시작...")
    
    # 1. 파라미터 (기존과 동일)
    par = Param2D(xmax=12.0, res=128, dt=0.001, timesteps=5000, m=1.0, im_time=True)
    V = potential_sho(par, omega=1.0)
    g = 500.0 
    
    # 2. 초기화
    wfc_offset = (0.0, 0.0) 
    
    # 'square' mode
    wfc_initial, Uk_full, g = init_state_and_ops(par, wfc_offset, g, initial_type='square')
    
    # 3. 시뮬레이션 실행
    wfc_final = run_simulation(par, wfc_initial, Uk_full, V, g)
    
    # 4. 정량 비교 (결과는 똑같이 TF 이론과 일치해야 함)
    if par.im_time:
        calculate_and_plot_tf_comparison(par, wfc_final, V, g)
    
    print("시뮬레이션 완료.")

if __name__ == "__main__":
    main4()

2D GPE 시뮬레이션 (Imaginary Time - Random Start) 시작...
Running frame 0 (Physics step 0)
Running frame 0 (Physics step 0)
Running frame 2 (Physics step 100)
Running frame 4 (Physics step 200)
Running frame 6 (Physics step 300)
Running frame 8 (Physics step 400)
Running frame 10 (Physics step 500)
Running frame 12 (Physics step 600)
Running frame 14 (Physics step 700)
Running frame 16 (Physics step 800)

--- 정량 분석 시작: 토마스-페르미(TF) 비교 ---
  >> 분석 결과 (g=500.0):
  >> RMSE (절대 오차): 0.000661
  >> Relative Error (불일치도): 5.5470 %
  >> Consistency (일치도): 94.4530 %
--- 정량 분석 완료 ---
시뮬레이션 완료.


Double well real time simulation

In [12]:
def main_tunneling():
    """터널링 시뮬레이션 메인 함수"""
    
    print("2D GPE 시뮬레이션 (Tunneling) 시작...")
    print("왼쪽 우물의 파동함수가 장벽을 뚫고 오른쪽으로 이동하는지 확인하세요.")
    
    # --- 1. 파라미터 설정 ---
    # 터널링은 시간이 좀 걸리므로 timesteps를 넉넉히 10000으로 잡습니다.
    # dt=0.001로 안정성 확보
    par = Param2D(xmax=6.0, res=128, dt=0.001, timesteps=200000, m=1.0, im_time=False)
    
    # --- 2. 이중 우물 퍼텐셜 생성 ---
    # barrier_height: 장벽 높이 (너무 높으면 터널링이 안 되고, 너무 낮으면 그냥 넘어감)
    # 40.0 정도가 시각적으로 적당합니다.
    V = potential_double_well(par, omega=1.0, barrier_height=10.0, barrier_width=0.5)
    
    # --- 3. 초기 상태 설정 ---
    g = 0.0 # 일단 g=0 (단일 입자 터널링)으로 깨끗한 진동을 봅니다.
    
    # (중요!) 파동함수를 중앙(0,0)이 아닌 '왼쪽 우물(-2.0, 0.0)'에서 시작합니다.
    wfc_offset = (-2.0, 0.0) 
    
    wfc_initial, Uk_full, g = init_state_and_ops(par, wfc_offset, g)
    
    # --- 4. 시뮬레이션 실행 ---
    # 50배속으로 실행 (터널링 주기가 길 수 있음)
    wfc_final = run_simulation(par, wfc_initial, Uk_full, V, g)
    
    print("시뮬레이션 완료.")

# 실행 부분 교체
if __name__ == "__main__":
    main_tunneling()

2D GPE 시뮬레이션 (Tunneling) 시작...
왼쪽 우물의 파동함수가 장벽을 뚫고 오른쪽으로 이동하는지 확인하세요.
Running frame 0 (Physics step 0)
Running frame 0 (Physics step 0)
Running frame 2 (Physics step 100)
Running frame 4 (Physics step 200)
Running frame 6 (Physics step 300)
Running frame 8 (Physics step 400)
Running frame 10 (Physics step 500)
Running frame 12 (Physics step 600)
Running frame 14 (Physics step 700)
Running frame 16 (Physics step 800)
Running frame 18 (Physics step 900)
Running frame 20 (Physics step 1000)
Running frame 22 (Physics step 1100)
Running frame 24 (Physics step 1200)
Running frame 26 (Physics step 1300)
Running frame 28 (Physics step 1400)
Running frame 30 (Physics step 1500)
시뮬레이션 완료.
